In [1]:
import os
import pandas as pd
import csv

### Data Preparation

Formatting the text file of original data to ".tsv" (similar to csv, but "/t" tab seperated file).


In [2]:
raw_data_path = "/kaggle/input/dialogieactclassificationgoodai/data_summarization_labeled.txt"
f = open(raw_data_path, "r")
# print(f.read())

prepared_data_dir = "data/prepared_data/"

utterances_data = os.path.join(prepared_data_dir, "full_utterances.tsv")
speakers_data = os.path.join(prepared_data_dir,"full_speakers.tsv")
labels_data = os.path.join(prepared_data_dir,"full_labels.tsv")
loss_mask_data = os.path.join(prepared_data_dir,"full_loss.tsv")
label_phrases_data = os.path.join(prepared_data_dir,"full_labels_phrases.tsv")

os.makedirs(prepared_data_dir, exist_ok=True)

Lines = f.readlines()

dialogue_count = 0

utterances = [f"Conv-{dialogue_count}"]
speakers = [f"Conv-{dialogue_count}"]
labels = [f"Conv-{dialogue_count}"]
loss = [f"Conv-{dialogue_count}"]
label_phrases = [f"Conv-{dialogue_count}"]

speaker_mapping = {"AgentName" : 0, "PlayerName": 1}

label_mapping = {"unknown": 0,\
                 "none": 1, \
                 "refuse": 2,\
                 "accept": 3,\
                 "interlude": 4,\
                 "standard": 5,\
                 "exchange": 6} # will add key(label):integer pair for every new label encountered 

for lines in Lines:
    
    lines_string = lines.strip()

    if lines_string == "":
        continue
    
    if lines_string.split(":")[0] == "Summary":
        
        label_encountered = lines_string.split(":")[-1][1:]

        if label_encountered not in label_mapping.keys():
            if label_encountered.split(" ")[0] == "exchange":
                labels[-1] = label_mapping["exchange"]
            else:
                labels[-1] = label_mapping["standard"]
        else:
            labels[-1] = label_mapping[label_encountered] # replacing last "unkown" label with Summary output
        
        label_phrases[-1] = label_encountered
        # print("got Summary", labels)

    elif lines.strip() == "###":
        # print(utterances)
        
        with open(utterances_data, 'a', newline='') as tsvfile:
            writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
            writer.writerow(utterances)

        with open(speakers_data, 'a', newline='') as tsvfile:
            writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
            writer.writerow(speakers)
        
        with open(labels_data, 'a', newline='') as tsvfile:
            writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
            writer.writerow(labels)
            
        with open(loss_mask_data, 'a', newline='') as tsvfile:
            writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
            writer.writerow(loss)
        
        with open(label_phrases_data, 'a', newline='') as tsvfile:
            writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
            writer.writerow(label_phrases)
        
        # utterances_df.loc[dialogue_count] = pd.Series(utterances)
        # speaker_df.loc[dialogue_count] = pd.Series(speakers)
        # labels_df.loc[dialogue_count] = pd.Series(labels)
        # loss_df.loc[dialogue_count] = pd.Series(loss)

        dialogue_count += 1
        
        utterances = [f"Conv-{dialogue_count}"]
        speakers = [f"Conv-{dialogue_count}"]
        labels = [f"Conv-{dialogue_count}"]
        loss = [f"Conv-{dialogue_count}"]
        label_phrases = [f"Conv-{dialogue_count}"]

        # print("End of dialogue.")

    else:
        
        utterances.append(lines_string.split("]")[-1][1:])

        for speaker_name, speaker_id in speaker_mapping.items():
            if (lines_string.split(";")[0][1:] == speaker_name):
                speakers.append(speaker_mapping[speaker_name])
        
        label_encountered = lines_string.split("]")[0].split(";")[-1][1:]
        
        if label_encountered not in label_mapping.keys():
            if label_encountered.split(" ")[0] == "exchange":
                labels.append(label_mapping["exchange"])
            else:
                labels.append(label_mapping["standard"])
        else:
            labels.append(label_mapping[label_encountered])
            
        label_phrases.append(label_encountered)
        


In [3]:
!cat data/prepared_data/full_labels_phrases.tsv

Conv-0	unknown	none	none	follow me	accept
Conv-1	unknown	none	give daughter to me	none	accept	none	none
Conv-2	unknown	accept	give medicine to me	follow me	none	none	none	follow me	accept
Conv-3	unknown	none	give gold to me
Conv-4	unknown	interlude	refuse	none	none	interlude	accept
Conv-5	unknown	interlude	refuse	interlude	refuse	interlude	interlude	interlude	refuse
Conv-6	unknown	none	exchange give gold to me for give dynamite to you	accept	give shovel to me	none	none	accept	intent give gold to you	intent give backyard key to you	unknown	none	none	pick god ingot and give dynamite to you	accept	interlude	refuse
Conv-7	unknown	interlude	refuse	interlude	refuse
Conv-8	unknown	none	none	accept	none	give medicine to me	none	interlude	refuse	none	none
Conv-9	unknown	none	exchange give supply crate to me for give key to you	none	exchange give crate to me for give key to you	interlude	refuse
Conv-10	unknown	none	none	give amelia to me	none
Conv-11	unknown	make torch
Conv-12	unknown	none	none


In [4]:
exchange_label_count = 0
standard_label_count = 0

for key, items in label_mapping.items():
    if key.split(" ")[0] == "exchange":
        exchange_label_count += 1
    elif key in ["none", "unknown", "interlude", "accept", "refuse"]:
        continue
    else:
        print(key)
        standard_label_count += 1

print(exchange_label_count)
print(standard_label_count)

standard
1
1


In [5]:
!cat data/prepared_data/full_labels.tsv

Conv-0	0	1	1	5	3
Conv-1	0	1	5	1	3	1	1
Conv-2	0	3	5	5	1	1	1	5	3
Conv-3	0	1	5
Conv-4	0	4	2	1	1	4	3
Conv-5	0	4	2	4	2	4	4	4	2
Conv-6	0	1	6	3	5	1	1	3	5	5	0	1	1	5	3	4	2
Conv-7	0	4	2	4	2
Conv-8	0	1	1	3	1	5	1	4	2	1	1
Conv-9	0	1	6	1	6	4	2
Conv-10	0	1	1	5	1
Conv-11	0	5
Conv-12	0	1	1
Conv-13	0	5	1	5	1
Conv-14	0	5	1	5	5
Conv-15	0	1	2	4	2	4	2
Conv-16	0	5	5	5	1	1	2
Conv-17	0	1	5	5	1	1	1
Conv-18	0	5	1	5	2
Conv-19	0	5	1	1	1	5	1
Conv-20	0	4	2	1	1	4	3	1	1
Conv-21	0	1	1	1
Conv-22	0	1	5
Conv-23	0	3	1	1	5	5	2	5	2	1	5
Conv-24	0	1	5	1
Conv-25	0	5	0	0	1	1	1	1
Conv-26	0	1	5	1	1
Conv-27	0	4	3	2	1	1	1
Conv-28	0	5	1	4	2	1	5	1	1	1	1	1	1
Conv-29	0	4	2	1	1	1	1	1	1	1	1	5	3
Conv-30	0	1	5	1	5	2	1	1	0	1	5	1	1	5	2	1	6	1	1	1	1	1	1	4	2	4	3	5	3	1	1
Conv-31	0	1	1
Conv-32	0	1	1	1	1
Conv-33	0	1	1	5	2	5	2	1	1	1	2
Conv-34	0	1	1	5	1	4	2	5	1	1	6	1	1
Conv-35	0	1	5	3	1	0	0	5	3
Conv-36	0	1	5	1	6	1	1	5	2	1	5	1	1	6	1	2	5
Conv-37	0	1	1	5	3	5	3
Conv-38	0	5
Conv-39	0	5
Conv-40	0	5
Conv-41	0	1	4
Conv-42	0	5
Conv-43	0	5
Conv-44	0	5	0	5	0	5


In [6]:
!cat data/prepared_data_dir/full_utterances.tsv

cat: data/prepared_data_dir/full_utterances.tsv: No such file or directory


In [7]:
# checking the distribution of various labels
sorted(label_mapping.items(), key=lambda x:x[1])

[('unknown', 0),
 ('none', 1),
 ('refuse', 2),
 ('accept', 3),
 ('interlude', 4),
 ('standard', 5),
 ('exchange', 6)]